In [1]:
{
    "tags": [
        "hide-cell"
    ]
}

### Install necessary libraries

try:
    import jax
except:
    # For cuda version, see https://github.com/google/jax#installation
    %pip install --upgrade "jax[cpu]" 
    import jax

try:
    import jsl
except:
    %pip install git+https://github.com/probml/jsl
    import jsl

try:
    import rich
except:
    %pip install rich
    import rich




     |████████████████████████████████| 946 kB 2.7 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 47.7 MB/s eta 0:00:01
     |████████████████████████████████| 17.6 MB 47.5 MB/s eta 0:00:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     |████████████████████████████████| 55.3 MB 73.1 MB/s eta 0:00:01
  Using cached typing_extensions-4.1.1-py3-none-any.whl (26 kB)
     |████████████████████████████████| 70.5 MB 723 kB/s  eta 0:00:01
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Created wheel for jax: filename=jax-0.3.5-py3-none-any.whl size=1095861 sha256=6886baa70817bbac3b5797b3720dcb81e49097f61cee7d2e1255823ea32ccad8
  Stored in directory: /Users/kpmurphy/Library/Caches/pip/wheels/05/30/aa/908988293721511b4b29e0aadf9b5d133d0f14f6c0a188e764
Successfully built jax
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/probml/jsl to /private/var/folders/mn/vt7cgfsx6zs9vblhvbbk7pf8003xtr/T/pip-req-buil

In [1]:
{
    "tags": [
        "hide-cell"
    ]
}


### Import standard libraries

import abc
from dataclasses import dataclass
import functools
import itertools

from typing import Any, Callable, NamedTuple, Optional, Union, Tuple

import matplotlib.pyplot as plt
import numpy as np


import jax
import jax.numpy as jnp
from jax import lax, vmap, jit, grad
from jax.scipy.special import logit
from jax.nn import softmax
from functools import partial
from jax.random import PRNGKey, split

import inspect
import inspect as py_inspect
from rich import inspect as r_inspect
from rich import print as r_print

def print_source(fname):
    r_print(py_inspect.getsource(fname))

ModuleNotFoundError: No module named 'rich'

(sec:ssm-intro)=
# What are State Space Models?


A state space model or SSM
is a partially observed Markov model,
in which the hidden state,  $z_t$,
evolves over time according to a Markov process.



```{figure} /figures/SSM-AR-inputs.png
:scale: 100%
:name: ssm-ar

Illustration of an SSM as a graphical model.
```

```{figure} /figures/SSM-simplified.png
:scale: 100%
:name: ssm-simplifed

Illustration of a simplified SSM.
```

(sec:casino-ex)=
## Example: Casino HMM

We first create the "Ocassionally dishonest casino" model from {cite}`Durbin98`.



There are 2 hidden states, each of which emit 6 possible observations.

In [5]:
# state transition matrix
A = np.array([
    [0.95, 0.05],
    [0.10, 0.90]
])

# observation matrix
B = np.array([
    [1/6, 1/6, 1/6, 1/6, 1/6, 1/6], # fair die
    [1/10, 1/10, 1/10, 1/10, 1/10, 5/10] # loaded die
])

pi, _ = normalize(np.array([1, 1]))
pi = np.array(pi)


(nstates, nobs) = np.shape(B)
